In [1]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv
import matplotlib.pyplot as plt
from cassandra.cluster import Cluster
import requests
from fuzzywuzzy import fuzz
from collections import defaultdict
from fuzzywuzzy import process
from nltk.tokenize import word_tokenize
from settrade_v2 import Investor
from pprint import pprint
from settrade_v2 import Investor
from datetime import datetime 

c:\Users\Vivobook\AppData\Local\Programs\Python\Python310\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
cluster = Cluster(['127.0.0.1']) 
session = cluster.connect()

In [ ]:
# สร้าง Keyspace หากยังไม่มี
session.execute("""
    CREATE KEYSPACE IF NOT EXISTS settrade_api 
    WITH replication = {'class': 'SimpleStrategy', 'replication_factor': 1}
""")

# ใช้ Keyspace ที่สร้างขึ้น
session.set_keyspace('settrade_api')

print("Keyspace 'settrade_api ' ถูกสร้างและใช้งานเรียบร้อย!")

Keyspace 'settrade_api ' ถูกสร้างและใช้งานเรียบร้อย!


In [ ]:
investor = Investor(
                    app_id="c5qta5OYvnmCP3Oz",
                    app_secret="AI/1vJ5OUqX9r0hSlAKl/8wcKwzhmO7G3aW/AF6yKSdK",
                    broker_id="SANDBOX",
                    app_code="SANDBOX",
                    is_auto_queue = False)

In [ ]:
# กรณี Investor
market = investor.MarketData()


res = market.get_candlestick(
    symbol="BBL",
    interval="1d",
    limit=1,
    normalized=True,
)

print(res)

{'lastSequence': 0, 'time': [1742144400], 'open': [146.0], 'high': [147.0], 'low': [144.5], 'close': [144.5], 'volume': [9769807], 'value': [1418735868.5]}


In [ ]:
print(res)
print(type(res))

{'lastSequence': 0, 'time': [1742144400], 'open': [146.0], 'high': [147.0], 'low': [144.5], 'close': [144.5], 'volume': [9769807], 'value': [1418735868.5]}
<class 'dict'>


In [ ]:
# 👉 เชื่อมต่อ Cassandra
cluster = Cluster(['127.0.0.1'])  # Localhost
session = cluster.connect()

# 👉 เลือกใช้ Keyspace
session.set_keyspace('settrade_api')

# 👉 ตรวจสอบและสร้างตาราง (ถ้ายังไม่มี)
session.execute("""
    CREATE TABLE IF NOT EXISTS candlestick_data (
        symbol TEXT,
        time TIMESTAMP PRIMARY KEY,
        open_price FLOAT,
        high_price FLOAT,
        low_price FLOAT,
        close_price FLOAT,
        volume INT,
        value FLOAT
    )
""")
print("✅ Keyspace และ Table พร้อมใช้งาน!")

# 👉 ฟังก์ชันดึงข้อมูลหุ้น
def fetch_and_store_stock(symbol="BBL", interval="1d", limit=10):
    res = market.get_candlestick(symbol=symbol, interval=interval, limit=limit, normalized=True)

    if not res:
        print(f"⚠️ ไม่พบข้อมูลสำหรับ {symbol}")
        return

    for i in range(len(res["time"])):
        timestamp = datetime.fromtimestamp(res["time"][i])  
        open_price = res["open"][i]
        high_price = res["high"][i]
        low_price = res["low"][i]
        close_price = res["close"][i]
        volume = res["volume"][i]
        value = res["value"][i]

        session.execute("""
            INSERT INTO candlestick_data (symbol, time, open_price, high_price, low_price, close_price, volume, value)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
        """, (symbol, timestamp, open_price, high_price, low_price, close_price, volume, value))

    print(f"✅ เพิ่มข้อมูล {len(res['time'])} รายการของหุ้น {symbol} สำเร็จ!")

# 👉 ทดสอบดึงข้อมูลหุ้น BBL
fetch_and_store_stock(symbol="BBL", interval="1d", limit=10)

✅ Keyspace และ Table พร้อมใช้งาน!
✅ เพิ่มข้อมูล 10 รายการของหุ้น BBL สำเร็จ!


In [ ]:
from cassandra.cluster import Cluster

# 👉 เชื่อมต่อ Cassandra
cluster = Cluster(['127.0.0.1'])  # ถ้าเป็น localhost
session = cluster.connect()

# 👉 เลือก Keyspace
session.set_keyspace('settrade_api')

# 👉 ดึงรายชื่อตารางทั้งหมดใน Keyspace
rows = session.execute("SELECT table_name FROM system_schema.tables WHERE keyspace_name = 'stock_data'")

print("📌 ตารางที่อยู่ใน Keyspace 'stock_data':")
for row in rows:
    print("-", row.table_name)

# 👉 เช็คโครงสร้างของตาราง candlestick_data
table_info = session.execute("SELECT * FROM system_schema.columns WHERE keyspace_name = 'stock_data' AND table_name = 'candlestick_data'")

print("\n📌 โครงสร้างของตาราง 'candlestick_data':")
for column in table_info:
    print("-", column.column_name, column.type)

📌 ตารางที่อยู่ใน Keyspace 'stock_data':

📌 โครงสร้างของตาราง 'candlestick_data':


In [ ]:
import pandas as pd
import time

# ✅ โหลดรายชื่อหุ้นทั้งหมดจากไฟล์ CSV
try:
    symbols_df = pd.read_csv("thai_stocks (1).csv")  # ไฟล์ต้องมีคอลัมน์ 'symbol'
    symbols = symbols_df['symbol'].dropna().tolist()
except FileNotFoundError:
    print("❌ ไม่พบไฟล์ 'thai_stocks.csv' ใช้รายชื่อหุ้นที่กำหนดแทน")
    symbols = ["PTT", "AOT", "SCB", "CPALL", "ADVANC"]

# ✅ ฟังก์ชันที่ดึงข้อมูล Financial Data สำหรับหุ้นแต่ละตัว
def insert_financial_data(symbol):
    try:
        # 👉 ดึงข้อมูลจาก Settrade API หรือแหล่งข้อมูลอื่น ๆ
        res = market.get_candlestick(symbol=symbol, interval="1d", limit=10, normalized=True)

        if not res:
            print(f"⚠️ ไม่พบข้อมูลสำหรับ {symbol}")
            return

        # 👉 เชื่อมต่อ Cassandra (หรือฐานข้อมูลอื่น ๆ)
        from cassandra.cluster import Cluster
        cluster = Cluster(['127.0.0.1'])  # ถ้าเป็น localhost
        session = cluster.connect()
        session.set_keyspace('settrade_api')

        # 👉 วนลูปเก็บข้อมูล
        for i in range(len(res["time"])):
            timestamp = datetime.fromtimestamp(res["time"][i])  
            open_price = res["open"][i]
            high_price = res["high"][i]
            low_price = res["low"][i]
            close_price = res["close"][i]
            volume = res["volume"][i]
            value = res["value"][i]

            # 👉 แทรกข้อมูลลงในตาราง
            session.execute("""
                INSERT INTO candlestick_data (symbol, time, open_price, high_price, low_price, close_price, volume, value)
                VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
            """, (symbol, timestamp, open_price, high_price, low_price, close_price, volume, value))

        print(f"✅ เพิ่มข้อมูล {len(res['time'])} รายการของหุ้น {symbol} สำเร็จ!")

    except Exception as e:     
        print(f"❌ เกิดข้อผิดพลาดในการดึงข้อมูลของหุ้น {symbol}: {e}")

# ✅ ดึงข้อมูล Financial Data สำหรับหุ้นทุกตัว
for symbol in symbols:
    
    insert_financial_data(symbol)
    time.sleep(2)  # ⏳ ควบคุม API Rate Limit

❌ เกิดข้อผิดพลาดในการดึงข้อมูลของหุ้น 24CS: The access token is invalid or has expired
❌ เกิดข้อผิดพลาดในการดึงข้อมูลของหุ้น 2S: The access token is invalid or has expired
❌ เกิดข้อผิดพลาดในการดึงข้อมูลของหุ้น 3BBIF: The access token is invalid or has expired
❌ เกิดข้อผิดพลาดในการดึงข้อมูลของหุ้น A: The access token is invalid or has expired
❌ เกิดข้อผิดพลาดในการดึงข้อมูลของหุ้น A5: The access token is invalid or has expired
❌ เกิดข้อผิดพลาดในการดึงข้อมูลของหุ้น AAI: The access token is invalid or has expired


KeyboardInterrupt: 

In [25]:
from cassandra.cluster import Cluster
import pandas as pd

# เชื่อมต่อกับ Cassandra
cluster = Cluster(['127.0.0.1'])  # ระบุ IP ของ Cassandra node ที่คุณใช้งาน
session = cluster.connect('settrade_api')  # เปลี่ยนเป็น keyspace ของคุณ

# สั่งเลือกข้อมูลจากตาราง candlestick_data
symbol = "ACAP"
query = f"SELECT time, open_price, high_price, low_price, close_price FROM candlestick_data WHERE symbol = '{symbol}' ALLOW FILTERING;"

# ดึงข้อมูลจาก Cassandra
rows = session.execute(query)

# สร้าง DataFrame จากผลลัพธ์
data = []
for row in rows:
    data.append({
        "time": row.time,
        "open_price": row.open_price,
        "high_price": row.high_price,
        "low_price": row.low_price,
        "close_price": row.close_price
    })

df = pd.DataFrame(data)

# แสดง DataFrame
print(df)

        time  open_price  high_price  low_price  close_price
0 2023-05-17        0.22        0.36       0.21         0.33
1 2023-05-15        0.46        0.47       0.41         0.41
2 2023-05-10        0.47        0.48       0.45         0.48
3 2023-05-09        0.47        0.48       0.46         0.48
4 2023-05-03        0.39        0.44       0.39         0.44
5 2023-05-12        0.50        0.50       0.46         0.48
6 2023-05-16        0.35        0.38       0.29         0.29
7 2023-05-11        0.47        0.49       0.46         0.49
8 2023-05-02        0.40        0.43       0.39         0.41
9 2023-05-08        0.41        0.50       0.41         0.48
